## 5.9 含并行连结的网络(GoogLeNet)
- Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., & Anguelov, D. & Rabinovich, A.(2015). Going deeper with convolutions. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1-9).
- GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。
- 在随后的几年里，研究人员对GoogLeNet进行了数次改进，本节介绍这个模型系列的第一个版本(v1)。

### 5.9.1 Inception块

- GoogLeNet中的基础卷积块叫做Inception块，得名于同名电影《盗梦空间》(Inception)。

![Inception块的结构](../img/inception.svg)

- 前3条线路使用窗口大小分别是$1\times1$、$3\times3$和$5\times5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times1$卷积来减少输入通道数，以降低模型复杂度。
- 第四条线路则使用$3\times3$最大池化层，后接$1\times1$卷积层来改变通道数。
- 4条线路都使用了合适的填充来使输入与输出的高和宽一致。
- 最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。
- Inception块中可以自定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [1]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

class Inception(nn.Block):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super().__init__(**kwargs)
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
        # 线路2, 1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
        self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')
        # 线路3, 1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
        self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')
        # 线路4， 3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
        self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')
        
    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return nd.concat(p1, p2, p3, p4, dim=1) # 在通道维上连结输出

### 5.9.2 GoogLeNet模型

- GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块(block)，每个模块之间使用步幅为2的$3\times3$最大池化层来减小输出高宽。

- 第一个模块使用一个64通道的$7\times7$卷积层。

In [2]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

- 第二个模块使用2个卷积层：首先是64通道的$1\times1$卷积层，然后是将通道增大3倍的$3\times3$卷积层。

In [3]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
       nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

- 第三个模块串联2个完整的Inception块。

In [4]:
b3 = nn.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32),
       Inception(128, (128, 192), (32, 96), 64),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

- 第四个模块更加复杂。
- 第二、第三条线路都会先按比例减小通道数。

In [5]:
b4 = nn.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64),
       Inception(160, (112, 224), (24, 64), 64),
       Inception(128, (128, 256), (24, 64), 64),
       Inception(112, (144, 288), (32, 64), 64),
       Inception(256, (160, 320), (32, 128), 128),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

- 第五个模块有两个Inception块。
- 需要注意的是，第五个模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。

In [6]:
b5 = nn.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128),
       Inception(384, (192, 384), (48, 128), 128),
       nn.GlobalAvgPool2D())

In [7]:
net = nn.Sequential()
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

- GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。

In [8]:
X = nd.random_uniform(shape=(1, 1, 96, 96))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential0 output shape:	 (1, 64, 24, 24)
sequential1 output shape:	 (1, 192, 12, 12)
sequential2 output shape:	 (1, 480, 6, 6)
sequential3 output shape:	 (1, 832, 3, 3)
sequential4 output shape:	 (1, 1024, 1, 1)
dense0 output shape:	 (1, 10)


### 5.9.3 训练模型

In [9]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

TypeError: 'function' object is not iterable

### 小结

- Inception块相当于一个有4条线路的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用$1\times1$卷积层减少输出通道数从而降低模型复杂度。
- GoogLeNet将多个设计精细的Inception块和其它层串联起来。其中Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的。